In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "26034001" # 店舗ID
SPREADSHEET_KEY = '1_q3Qe0lbBob0YwJ9tdhsTxmvhbgwmAYWNDYDXINZml4' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 641 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/19', '025670', '1', '35385.0', '4214.0', '57', '9']
['08/19', '025670', '2', '35128.0', '14144.0', '76', '7']
['08/19', '025670', '3', '38974.0', '-16550.0', '28', '7']
['08/19', '025670', '4', '35128.0', '603.0', '56', '5']
['08/19', '025814', '5', '14872.0', '5418.0', '14', '4']
['08/19', '025814', '6', '20513.0', '-5115.0', '10', '2']
['08/19', '025775', '7', '14359.0', '-10531.0', '2', '2']
['08/19', '025775', '8', '17949.0', '3010.0', '11', '3']
['08/19', '025806', '9', '13333.0', '-1203.0', '7', '3']
['08/19', '025806', '10', '15897.0', '11135.0', '16', '3']
['08/19', '025806', '11', '15897.0', '-10832.0', '1', '1']
['08/19', '025593', '12', '13077.0', '603.0', '9', '2']
['08/19', '025852', '13', '8974.0', '-1805.0', '8', '2']
['08/19', '025852', '14', '11282.0', '-6619.0', '4', '1']
['08/19', '025852', '15', '5641.0', '-4814.0', '0', '0']
['08/19', '025852', '16', '7949.0', '3010.0', '12', '3']
['08/19', '025550', '17', '26154.0', '-8425.0', '7', '3']
['08/19', '025495', '1

['08/19', '025579', '140', '41282.0', '1806.0', '22', '5']
['08/19', '025779', '141', '35385.0', '-14744.0', '19', '6']
['08/19', '025779', '142', '24103.0', '16853.0', '40', '7']
['08/19', '025750', '143', '28718.0', '-2106.0', '57', '5']
['08/19', '025750', '144', '24359.0', '-7823.0', '29', '3']
['08/19', '025750', '145', '30513.0', '-12638.0', '34', '5']
['08/19', '025803', '146', '10000.0', '-1504.0', '5', '1']
['08/19', '025803', '147', '17436.0', '-3911.0', '7', '1']
['08/19', '025638', '148', '25385.0', '-1504.0', '17', '6']
['08/19', '025792', '149', '20769.0', '13543.0', '21', '4']
['08/19', '025483', '150', '37949.0', '-14143.0', '14', '3']
['08/19', '025483', '151', '26410.0', '16251.0', '31', '4']
['08/19', '025483', '152', '21026.0', '8427.0', '25', '5']
['08/19', '025483', '153', '28205.0', '-3911.0', '16', '3']
['08/19', '025796', '154', '20769.0', '-3911.0', '9', '3']
['08/19', '025796', '155', '9744.0', '-5115.0', '2', '2']
['08/19', '025688', '156', '25128.0', '8126.

['08/19', '025605', '277', '18205.0', '2709.0', '24', '8']
['08/19', '025605', '278', '7179.0', '-3008.0', '4', '1']
['08/19', '025605', '279', '15641.0', '-902.0', '15', '8']
['08/19', '025605', '280', '13846.0', '3311.0', '19', '5']
['08/19', '025521', '281', '18718.0', '3010.0', '24', '7']
['08/19', '025521', '282', '44103.0', '6019.0', '56', '22']
['08/19', '025521', '283', '17949.0', '-1203.0', '19', '8']
['08/19', '025521', '284', '21538.0', '-1805.0', '20', '9']
['08/19', '025521', '285', '6410.0', '-1203.0', '6', '2']
['08/19', '025521', '286', '22821.0', '-4212.0', '20', '9']
['08/19', '025817', '287', '23333.0', '-4513.0', '43', '6']
['08/19', '025817', '288', '40769.0', '14144.0', '99', '29']
['08/19', '025817', '289', '14615.0', '12339.0', '75', '6']
['08/19', '025817', '290', '29744.0', '-2106.0', '74', '16']
['08/19', '025737', '291', '12564.0', '-4814.0', '13', '3']
['08/19', '025737', '292', '10256.0', '-4212.0', '9', '3']
['08/19', '025737', '293', '10513.0', '-5115.0'

['08/19', '025895', '419', '35641.0', '-17754.0', '8', '4']
['08/19', '025895', '420', '38718.0', '-15346.0', '14', '8']
['08/19', '025818', '421', '18205.0', '-10832.0', '3', '1']
['08/19', '025818', '422', '2564.0', '-1805.0', '1', '1']
['08/19', '025818', '423', '0', '0', '0', '0']
['08/19', '025818', '424', '769.0', '-1504.0', '0', '0']
['08/19', '025818', '425', '3333.0', '6320.0', '6', '1']
['08/19', '025826', '426', '0', '0', '0', '0']
['08/19', '025826', '427', '769.0', '-1203.0', '0', '0']
['08/19', '025826', '428', '0', '0', '0', '0']
['08/19', '025826', '429', '0', '0', '0', '0']
['08/19', '025826', '430', '8974.0', '-7522.0', '0', '0']
['08/19', '025849', '431', '2564.0', '-902.0', '2', '1']
['08/19', '025849', '432', '4615.0', '-3911.0', '0', '0']
['08/19', '025849', '433', '1795.0', '-1504.0', '0', '0']
['08/19', '025849', '434', '7436.0', '-4814.0', '1', '1']
['08/19', '025849', '435', '0', '0', '0', '0']
['08/19', '025680', '436', '513.0', '-902.0', '0', '0']
['08/19', 

['08/19', '025864', '560', '9231.0', '4214.0', '16', '3']
['08/19', '025761', '561', '2564.0', '-2106.0', '0', '0']
['08/19', '025761', '562', '4103.0', '-3610.0', '0', '0']
['08/19', '025761', '563', '8718.0', '-6920.0', '3', '1']
['08/19', '025761', '564', '5128.0', '-4513.0', '0', '0']
['08/19', '025665', '565', '40513.0', '-7823.0', '17', '5']
['08/19', '025665', '566', '35128.0', '64098.0', '63', '8']
['08/19', '025665', '567', '37179.0', '15348.0', '29', '4']
['08/19', '025665', '568', '43590.0', '16552.0', '36', '7']
['08/19', '025665', '569', '32051.0', '12038.0', '27', '5']
['08/19', '025665', '570', '46923.0', '-4814.0', '25', '11']
['08/19', '025665', '571', '37179.0', '34607.0', '45', '8']
['08/19', '025665', '572', '50256.0', '-15948.0', '18', '7']
['08/19', '025665', '573', '37692.0', '15047.0', '31', '4']
['08/19', '025665', '574', '32821.0', '47547.0', '49', '6']
['08/19', '025665', '575', '41282.0', '4816.0', '26', '5']
['08/19', '025665', '576', '39231.0', '8728.0', '

['08/19', '119996', '683', '6312', '-278.0', '23', '17', '--']
['08/19', '119996', '684', '4441', '-93.0', '17', '12', '--']
['08/19', '119996', '685', '4102', '-1065.0', '11', '9', '--']
['08/19', '119996', '686', '5063', '-1019.0', '16', '12', '--']
['08/19', '119996', '687', '3035', '-231.0', '12', '6', '--']
['08/19', '119996', '688', '3009', '139.0', '10', '11', '--']
['08/19', '119996', '689', '4096', '602.0', '18', '10', '--']
['08/19', '119996', '690', '3581', '-231.0', '14', '7', '--']
['08/19', '119996', '691', '3102', '231.0', '14', '4', '--']
['08/19', '119996', '692', '6080', '1157.0', '26', '20', '--']
['08/19', '119916', '693', '3606', '-648.0', '11', '12', '--']
['08/19', '119916', '694', '6904', '-370.0', '22', '24', '--']
['08/19', '119916', '695', '5148', '139.0', '17', '19', '--']
['08/19', '119916', '696', '3393', '231.0', '14', '8', '--']
['08/19', '119916', '697', '4901', '-509.0', '17', '13', '--']
['08/19', '120127', '698', '5583', '2546.0', '0', '1', '12']
['0

['08/19', '120010', '819', '6573', '1111.0', '27', '25', '--']
['08/19', '120010', '820', '6362', '1528.0', '28', '20', '--']
['08/19', '120010', '821', '4131', '-1296.0', '9', '10', '--']
['08/19', '120010', '822', '5636', '-1481.0', '15', '17', '--']
['08/19', '120010', '823', '5346', '-2778.0', '11', '13', '--']
['08/19', '120010', '824', '7844', '1250.0', '33', '27', '--']
['08/19', '120010', '825', '6955', '1157.0', '31', '19', '--']
['08/19', '120010', '826', '4165', '-1713.0', '9', '12', '--']
['08/19', '120010', '827', '6177', '-972.0', '19', '15', '--']
['08/19', '120010', '828', '4825', '-694.0', '17', '12', '--']
['08/19', '120010', '829', '7470', '231.0', '28', '24', '--']
['08/19', '120010', '830', '6103', '-463.0', '23', '9', '--']
['08/19', '120010', '831', '5826', '-278.0', '20', '19', '--']
['08/19', '120010', '832', '8403', '-185.0', '24', '36', '--']
['08/19', '120010', '833', '4541', '-602.0', '14', '14', '--']
['08/19', '120010', '834', '5887', '-1435.0', '20', '11

['08/19', '120103', '956', '1102', '-1111.0', '1', '0', '4']
['08/19', '120103', '957', '1775', '-1250.0', '9', '3', '4']
['08/19', '120103', '958', '933', '-463.0', '4', '1', '2']
['08/19', '120103', '959', '3979', '-1157.0', '14', '4', '16']
['08/19', '120103', '960', '2317', '-1157.0', '7', '3', '8']
['08/19', '120103', '961', '2419', '-1481.0', '6', '2', '8']
['08/19', '120103', '962', '397', '-185.0', '3', '1', '1']
['08/19', '120103', '963', '2952', '-1481.0', '10', '2', '9']
['08/19', '120014', '964', '525', '-463.0', '2', '1', '--']
['08/19', '119990', '965', '0', '0', '0', '0', '--']
['08/19', '119897', '966', '2213', '-602.0', '16', '0', '8']
['08/19', '119897', '967', '49', '0', '0', '0', '--']
['08/19', '119897', '968', '774', '-509.0', '6', '0', '2']
['08/19', '119987', '969', '704', '-787.0', '0', '0', '2']
['08/19', '119987', '970', '0', '0', '0', '0', '--']
['08/19', '119864', '971', '6150', '-139.0', '25', '0', '17']
['08/19', '119864', '972', '1029', '-556.0', '3', '0

['08/19', '120104', '1091', '253', '-370.0', '0', '0', '--']
['08/19', '120104', '1092', '293', '93.0', '0', '0', '2']
['08/19', '120104', '1093', '0', '0', '0', '0', '--']
['08/19', '120104', '1094', '4735', '2130.0', '0', '9', '24']
['08/19', '120104', '1095', '173', '0', '0', '0', '1']
['08/19', '120104', '1096', '742', '-648.0', '0', '0', '2']


{'spreadsheetId': '1_q3Qe0lbBob0YwJ9tdhsTxmvhbgwmAYWNDYDXINZml4',
 'updates': {'spreadsheetId': '1_q3Qe0lbBob0YwJ9tdhsTxmvhbgwmAYWNDYDXINZml4',
  'updatedRange': 'S5RAW!F6234:M6271',
  'updatedRows': 38,
  'updatedColumns': 8,
  'updatedCells': 304}}